In [1]:
%matplotlib inline

import pystan 
import pickle

#Stan fitting


In [2]:
model = """
data {
    int<lower=0> n;
    real<lower=0> V;
    real<lower=0> S;
    vector[n] t;
    vector[n] LogN;
    vector[n] LogNdose;
}

transformed data{
    vector[n] Ndose;
    for (i in 1:n){
        Ndose[i] = 10^LogNdose[i];
    }
}

parameters { 
    real<lower=0, upper= 10> LogNmax; 
    real<lower=-10, upper= 0> Logmu; 
    real<lower=0> sigma;
}

transformed parameters {
    vector[n] LogPredicted;
    real<lower=0> mu;
    real<lower=0> Nmax;
    mu = 10^Logmu;
    Nmax = 10^LogNmax;
    for (i in 1:n){
        LogPredicted[i] = log10(V*Ndose[i]*Nmax*S*(exp(mu*Ndose[i]*t[i])-exp(mu*Nmax*S*t[i]/V))/(V*Ndose[i]*exp(mu*Ndose[i]*t[i])-Nmax*S*exp(mu*Nmax*S*t[i]/V)));
    }
}

model {
    for (i in 1:n){
        LogN[i] ~ normal(LogPredicted[i], sigma);
    }
}
"""
sm = pystan.StanModel(model_code=model)

with open('new stan model.pkl', 'wb') as f:

    pickle.dump(sm, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9981b91acc61f17b4ba9e6690c401580 NOW.
